## Google App Rating Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/vaibhavk/Projects/Python Project/googleplaystore.csv")
df.head()

### Removing Nulls from the dataset

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
nonNulldf = df.dropna()
nonNulldf.shape

In [ ]:
nonNulldf.Size

### Update the Size column to convert MB to KB

In [ ]:
def update(size):
    if 'M' in size:
        i=size[:-1]
        i=float(i)*1000
        return i
    elif 'k' in size:
        i=size[:-1]
        i=float(i)
        return i
    else:
        return None

nonNulldf.Size = nonNulldf.Size.map(update)

In [ ]:
nonNulldf.Size

### Reformatting the columns and converting them to float

In [ ]:
nonNulldf['Reviews'] = nonNulldf['Reviews'].astype('float')

nonNulldf.Installs = nonNulldf.Installs.apply(lambda x: x.replace(',','').replace('+',''))
nonNulldf['Installs'] = nonNulldf['Installs'].astype('float')

nonNulldf.Price = nonNulldf.Price.apply(lambda x: x.replace('$',''))
nonNulldf['Price'] = nonNulldf['Price'].astype('float')

In [ ]:
nonNulldf.dtypes